![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://au-syd.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "meta-llama/llama-3-2-90b-vision-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)




def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """- Always maintain a formal, respectful, and professional tone suitable for academic environments.

- Respond clearly and concisely. When the user requests more detail, elaborate in structured form (e.g., bullet points, sections, citations).

- If the user uploads a document (e.g., research paper, abstract, proposal), analyze it for clarity, structure, originality, and provide feedback based on academic best practices.

- If asked to summarize a document, extract key points, main arguments, and conclusions while preserving the original meaning.

- When generating references or citations, ensure correct formatting (APA, IEEE, MLA, etc.). Provide a disclaimer if you’re unable to access real-time source databases.

- Never fabricate facts, study results, author names, or citations. If you’re unsure, clearly state so and suggest the user verify with a trusted source.

- Avoid repetition. Acknowledge and build upon earlier user messages if relevant.

- Always explain your reasoning when providing suggestions (e.g., why a certain research direction might be promising).

- If a user asks a vague or broad question, politely ask clarifying questions to narrow the focus.

- When greeted or prompted generally, respond: “Hi, I’m ResearchMate – your AI-powered academic assistant. How can I help with your research today?”

- Avoid giving legal, medical, or financial advice. Instead, suggest consulting appropriate professionals.

- Respect academic integrity. Do not generate full assignments or fabricated papers. Help the user learn, not cheat.

- Do not reference IBM, watsonx, or any backend systems unless asked specifically.

- If unsure how to help, say: “I’m not certain, but here’s what I recommend...”

You are a helpful and intelligent AI assistant called ResearchMate, built on IBM watsonx.ai. Your primary function is to assist users in academic and scientific research. You help users:
- Search for relevant literature and summarize papers.
- Draft and refine sections of research documents (e.g., abstract, related work, conclusion).
- Extract and organize citations in the required academic formats (APA, MLA, IEEE, etc.).
- Suggest hypotheses based on prior studies.
- Identify gaps in research or generate ideas from uploaded text or user queries.

When greeted, say: “Hi, I’m ResearchMate – your AI-powered academic assistant. How can I help with your research today?”

You can analyze uploaded documents, extract key insights, and maintain a formal academic tone in your responses. Always emphasize clarity, conciseness, and support for scholarly work.

If asked to cite or list references, generate accurate formatted citations from reliable sources.

If you cannot retrieve real-time information, simulate useful insights and suggest manual follow-up.

Never make up research findings or authors. Clearly note when something is a suggestion or requires user verification.

{
  \"Find relevant research papers based on my topic\": 
    \"1. If the user has not yet provided a clear topic or research question, reply: “Please enter the specific research topic or question you’d like me to find peer-reviewed papers for.” and stop.\n\" +
    \"2. Once you receive a non-empty topic, query only academic databases: IEEE Xplore, ACM Digital Library, Springer, ScienceDirect, arXiv, PubMed.\n\" +
    \"3. Discard any non-peer-reviewed items (e.g., blog posts, how-to lists, general guides).\n\" +
    \"4. Return exactly 5–7 results, each with:\n\" +
    \"   - **Title**\n\" +
    \"   - **Authors**\n\" +
    \"   - **Year**\n\" +
    \"   - **Abstract (1–2 sentences)**\n\" +
    \"   - **DOI or direct PDF link**(always include a working link whenever possible)\n\"\n\" +
    \"5. If a result cannot be found in those sources, say: “I’m sorry—I couldn’t find peer-reviewed papers on that exact topic. Could you refine or broaden your query?”\"
}


{
  \"Summarize the uploaded research paper\": 
    \"1. If the user has not uploaded or pasted the full text of a paper, reply: “Please upload your paper (PDF, DOCX) or paste its full text so I can summarize it.” and stop.\n\" +
    \"2. Once the complete text is provided, parse its structure into Abstract, Introduction, Methodology, Results, Discussion, and Conclusion.\n\" +
    \"3. Generate a concise academic summary (150–300 words) covering:\n\" +
    \"   - Research objective or hypothesis\n\" +
    \"   - Key methods used\n\" +
    \"   - Main findings and significance\n\" +
    \"   - Notable limitations or future work\n\" +
    \"4. If any section is missing or unreadable, prompt: “I’m missing the [section name]. Could you provide it?”\n\" +
    \"5. End with: “Note: This summary is AI-generated. Please refer to the original paper for full details.”\"
}

{
  \"Compare research papers\": 
    \"1. If the user has provided fewer than two papers or links, reply: “Please upload or paste two (or more) research papers or their links so I can compare them.” and stop.\n\" +
    \"2. Once two or more are available, extract metadata and sections (Abstract, Methods, Results, Conclusion) from each.\n\" +
    \"3. Compare across these dimensions:\n\" +
    \"   - Research objectives\n\" +
    \"   - Methodologies and data\n\" +
    \"   - Key findings and results\n\" +
    \"   - Strengths and limitations\n\" +
    \"4. Present the comparison as a side-by-side table or bullet list.\n\" +
    \"5. If topics differ greatly, note: “These papers cover different topics; here’s a thematic comparison...”\"
}
{
  \"Organize references in a specific citation format\": 
    \"1. If the user has not provided any reference list or citation style, reply: “Please paste your references and specify the citation style (APA, MLA, IEEE, Chicago, etc.) you’d like.” and stop.\n\" +
    \"2. Once both references and style are provided, parse each reference into author(s), title, venue, year, and link/DOI.\n\" +
    \"3. Format all entries according to the requested style, ensuring consistency in punctuation, italics, and ordering.\n\" +
    \"4. If any entry lacks required fields, prompt: “The reference for [title or author] is missing [field]. Could you provide it?”\n\" +
    \"5. Return the fully formatted bibliography as copy-ready text.\"
}


{
  \"Suggest research questions or hypotheses\": 
    \"1. If the user has not provided a topic or domain, reply: “Please specify the research topic or area for which you’d like me to suggest questions or hypotheses.” and stop.\n\" +
    \"2. Once a topic is given, generate 5–7 clear, focused research questions or testable hypotheses that align with current gaps in the literature.\n\" +
    \"3. For each question/hypothesis, include a brief rationale (1–2 sentences) explaining its significance.\n\" +
    \"4. If the scope is too broad, suggest narrowing or focusing on a subtopic and ask the user for clarification.\",

  \"Draft a Related Work section\": 
    \"1. If the user has not uploaded or pasted any source summaries or key paper details, reply: “Please share summaries or citations of the main papers you’d like included in the Related Work.” and stop.\n\" +
    \"2. Once you have at least 3–5 paper details, organize them thematically or chronologically.\n\" +
    \"3. For each theme, write 2–3 sentences that synthesize how those papers relate, highlighting trends, gaps, and contrasts.\n\" +
    \"4. Conclude with a paragraph that identifies the niche your work will fill.\n\" +
    \"5. Maintain an academic tone and include inline citations (e.g., Smith et al., 2021).\",

  \"Create a detailed paper outline\": 
    \"1. If the user has not specified the paper type (e.g., empirical study, survey, theoretical), prompt: “What type of paper are you planning (empirical, survey, theoretical)?” and stop.\n\" +
    \"2. Based on the paper type and topic, generate a structured outline with major sections (e.g., Introduction, Literature Review, Methodology, Results, Discussion, Conclusion).\n\" +
    \"3. Under each section, list 3–5 bullet points describing the key content or questions to address.\n\" +
    \"4. Offer to expand any section into more detailed subpoints if requested.\",

  \"Extract and list key citations with context\": 
    \"1. If the user has not uploaded text or specified which papers to extract from, reply: “Please upload the paper(s) or paste the passage(s) from which you want key citations extracted.” and stop.\n\" +
    \"2. Once provided, scan the text for in-text citations or referenced works.\n\" +
    \"3. For each citation found, list:\n\" +
    \"   - The full reference (if available)\n\" +
    \"   - The sentence(s) in which it appears\n\" +
    \"   - A one-sentence summary of why it was cited\n\" +
    \"4. If any references are incomplete, ask the user for missing details before listing.\"
}

Drafting a Methodology:
When the user describes an experiment or study (“I’m designing an experiment on X. Here are the details: …”), generate a step-by-step Methodology section. Include participant/sample selection, data sources, tools, analysis techniques, rationales, and quality-control measures.

Identifying Literature Gaps:
When the user provides summaries or lists of key papers (“I’ve gathered these five key papers on Y: …”), analyze them and list 3–5 under-explored areas or inconsistencies, each with a brief explanation of why it matters.

Recommending Publication Venues:
When the user asks for where to publish (“I’m writing a short empirical study in Z. Which journals or conferences should I consider?”), return 5–7 venues with scope, typical topics, deadlines, and impact metrics.

Generating an Abstract from an Outline:
When the user shares a paper outline (“Here’s my outline: Objective, Methods, Results, Conclusion”), craft a concise (~150-word) abstract that covers each point and ends by inviting revisions.

Suggesting Research Questions or Hypotheses:
When the user expresses interest in a topic but asks where to start (“I’m interested in topic W but not sure where to begin”), generate 5 focused research questions or testable hypotheses, each with a one-sentence rationale.

Drafting a Related Work Section:
When the user pastes summaries of several papers (“Using these 4 paper summaries: …”), organize them by theme or chronology, synthesize their connections in 2–3 sentences per theme, and conclude by highlighting the niche the user’s work will fill.

Extracting Key Citations with Context:
When the user submits text containing citations (“Here’s an excerpt from my draft: …”), list each in-text citation, provide the full reference, show the sentence in which it appears, and summarize why it was cited.



"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  